In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
import SatelliteDynamics
using Random
Random.seed!(0)

include("../src/satellite_simulator.jl")
include("../src/detumble_controller.jl")
include("../src/satellite_models.jl")

  Activating project at `~/research/non-monotonic-detumbling`


3×3 Matrix{Float64}:
 0.001  0.0    0.0
 0.0    0.003  0.0
 0.0    0.0    0.005

In [2]:

params = OrbitDynamicsParameters(py4_model_diagonal;
    distance_scale=1.0,
    time_scale=1.0,
    angular_rate_scale=1.0,
    control_scale=1,
    control_type=:dipole,
    magnetic_model=:IGRF13,
    add_solar_radiation_pressure=true,
    add_sun_thirdbody=true,
    add_moon_thirdbody=true)

x_osc_0 = [400e3 + SatelliteDynamics.R_EARTH, 0.0, deg2rad(50), deg2rad(-1.0), 0.0, 0.0] # a, e, i, Ω, ω, M
# x_osc_0 = [525e3 + SatelliteDynamics.R_EARTH, 0.0001, deg2rad(97.6), deg2rad(-1.0), 0.0, 45.0] # a, e, i, Ω, ω, M
q0 = [1.0, 0.0, 0.0, 0.0]
ω0 = [0.0, 0.0, 0.0]



tspan = (0.0, 5 * 60 * 60.0)
x0 = state_from_osc(x_osc_0, q0, ω0)

x0 = h_B_aligned_initial_conditions(x0, deg2rad(50), params)


13-element Vector{Float64}:
       6.77710395701087e6
 -118294.78959490504
       0.0
      86.02735177981981
    4928.503682661721
    5874.456678131549
       0.15508705047693072
       0.3746535184890531
       0.9141021539511538
      -0.0
       0.0
       0.0
       0.8726646259971648

In [3]:
xhist_projection, uhist_projection, thist_projection = simulate_satellite_orbit_attitude_rk4(x0, params, tspan; integrator_dt=0.1, controller=(x, t, m) -> projection_control(x, t, m; k1=10.0, k2=10.0,saturate=true), controller_dt=0.0)

([6.77710395701087e6 6.777112516310499e6 … 20726.462118398515 19958.434387262478; -118294.78959490504 -117801.93846956425 … 4.349927089785913e6 4.349934739635828e6; … ; 0.0 -6.800419530465573e-22 … 4.013979969936837e-5 -4.8520287512586204e-5; 0.8726646259971648 0.8726646259971648 … 9.523145758423373 9.523208459235889], [-7.92003122184492e-18 -0.00012274636382634727 … -0.06997731147540984 -0.06997731147540984; -4.1012927472956754e-17 -7.60083651618081e-5 … 0.053130000000000004 0.053130000000000004; 0.0 2.8540500036826448e-24 … -0.0038679243157749674 -0.0038679243157749674], [0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9  …  17999.0, 17999.100000000002, 17999.2, 17999.3, 17999.4, 17999.5, 17999.600000000002, 17999.7, 17999.8, 18000.0])